##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Escrevendo um loop de treinamento do zero

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/keras/writing_a_training_loop_from_scratch.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/keras/writing_a_training_loop_from_scratch.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/keras/writing_a_training_loop_from_scratch.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Configuração

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## Introdução

O Keras fornece loops padrão de treinamento e avaliação, `fit()` e `evaluate()`. Seu uso é abordado no guia [Treinamento e avaliação com métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate/).

Se você deseja personalizar o algoritmo de aprendizado de seu modelo enquanto ainda aproveita a conveniência de `fit()` (por exemplo, para treinar uma GAN usando `fit()`), você pode criar uma subclasse da classe `Model` e implementar seu próprio método `train_step()`, que é chamado repetidamente durante `fit()`. Isto é abordado no guia <a href="https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit/" data-md-type="link">Personalizando o que acontece em `fit()`</a>.

Agora, se você deseja um controle de nível muito baixo sobre o treinamento e avaliação, deve escrever seus próprios loops de treinamento e avaliação do zero. Este guia trata desse assunto.

## Usando o `GradientTape`: um primeiro exemplo completo

Chamar um modelo dentro de um escopo `GradientTape` permite recuperar os gradientes dos pesos treináveis ​​da camada em relação a um valor de perda. Usando uma instância do otimizador, você pode usar esses gradientes para atualizar essas variáveis ​​(que podem ser recuperadas usando `model.trainable_weights`).

Vamos considerar um modelo MNIST simples:

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

Vamos treiná-lo usando gradiente de minilote com um loop de treinamento personalizado.

Primeiro, vamos precisar de um otimizador, uma função de perda e um dataset:

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

Aqui está o nosso loop de treinamento:

- Abrimos um loop `for` que itera sobre épocas
- Para cada época, abrimos um loop `for` que itera sobre o dataset, em lotes
- Para cada lote, abrimos um escopo `GradientTape()`
- Dentro deste escopo, chamamos o modelo (forward pass) e calculamos a perda
- Fora do escopo, recuperamos os gradientes dos pesos do modelo em relação à perda
- Finalmente, usamos o otimizador para atualizar os pesos do modelo com base nos gradientes

In [ ]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))

## Tratamento de métricas em baixo nível

Vamos adicionar monitoramento de métricas a esse loop básico.

Você pode reutilizar prontamente as métricas integradas (ou personalizadas que você escreveu) em tais loops de treinamento escritos do zero. Aqui está o fluxo:

- Instancie a métrica no início do loop
- Chame `metric.update_state()` após cada lote
- Chame `metric.result()` quando precisar exibir o valor atual da métrica
- Chame `metric.reset_states()` quando precisar limpar o estado da métrica (normalmente no final de uma época)

Vamos usar esse conhecimento para calcular `SparseCategoricalAccuracy` em dados de validação no final de cada época:

In [ ]:
# Get model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

Eis aqui nosso loop de treinamento e avaliação:

In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

## Acelerando seu passo de treinamento com `tf.function`

O runtime padrão no TensorFlow 2 é [execução antecipada](https://www.tensorflow.org/guide/eager) (eager execution). Como tal, nosso loop de treinamento acima é executado imediatamente de forma "eager".

Isso é muito bom para a depuração, mas a compilação de grafos tem uma vantagem de desempenho definitiva. Descrever sua computação como um grafo estático permite que a estrutura aplique otimizações de desempenho global. Isto é impossível quando o framework é limitado a executar avidamente uma operação depois da outra, sem saber o que vem a seguir.

Você pode compilar num grafo estático qualquer função que receba tensores como entrada. Basta adicionar um decorador `@tf.function` nele, como mostrado a seguir:

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value


Vamos fazer o mesmo com o passo de avaliação:

In [ ]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)


Agora, vamos executar novamente nosso loop de treinamento com este passo de treinamento compilado:

In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

Muito mais rápido, não foi?

## Manipulação de perdas rastreadas pelo modelo em baixo nível

Camadas e modelos rastreiam recursivamente quaisquer perdas criadas durante o forward pass por camadas que chamam `self.add_loss(value)`. A lista resultante de valores de perda escalar está disponível através da propriedade `model.losses` no final do forward pass.

Se você quiser usar esses componentes de perda, você deve somá-los e adicioná-los à perda principal em seu passo de treinamento.

Considere a camada a seguir, que gera uma perda de regularização da atividade:

In [ ]:
class ActivityRegularizationLayer(layers.Layer):
    def call(self, inputs):
        self.add_loss(1e-2 * tf.reduce_sum(inputs))
        return inputs


Vamos construir um modelo bem simples que a utilize:

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu")(inputs)
# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

O nosso passo de treinamento deve agora ficar assim:

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
        # Add any extra losses created during the forward pass.
        loss_value += sum(model.losses)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value


## Resumo

Agora você sabe tudo o que há para saber sobre como usar loops de treinamento integrados e escrever seus próprios loops do zero.

Para concluir, eis um exemplo completo e simples que junta tudo o que você aprendeu neste guia: um DCGAN treinado em dígitos MNIST.

## Um exemplo completo: um loop de treinamento GAN do zero

Você talvez esteja familiarizado com Redes Adversárias Generativas (Generative Adversarial Networks), ou GANs. As GANs podem gerar novas imagens que parecem quase reais, aprendendo a distribuição latente de um dataset de treinamento de imagens (o "espaço latente" das imagens).

Uma GAN é composta de duas partes: um modelo "gerador" que mapeia pontos no espaço latente para pontos no espaço da imagem, um modelo "discriminador", um classificador que pode diferenciar entre imagens reais (do dataset de treinamento) e imagens falsas (a saída da rede do gerador).

Um loop de treinamento GAN tem a seguinte estrutura:

1. Treine o discriminador.

- Obtenha uma amostra de um lote de pontos aleatórios no espaço latente.
- Transforme os pontos em imagens falsas por meio do modelo "gerador".
- Obtenha um lote de imagens reais e combine-as com as imagens geradas.
- Treine o modelo "discriminador" para classificar imagens geradas versus imagens reais.

1. Treine o gerador.

- Obtenha uma amostra de pontos aleatórios no espaço latente.
- Transforme os pontos em imagens falsas através da rede "gerador".
- Obtenha um lote de imagens reais e combine-as com as imagens geradas.
- Treine o modelo "gerador" para "enganar" o discriminador e classificar as imagens falsas como reais.

Para uma visão geral muito mais detalhada de como as GANs funcionam, consulte [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) .

Vamos implementar este loop de treinamento. Primeiro, crie o discriminador destinado a classificar dígitos falsos vs. reais:

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)
discriminator.summary()

Em seguida criaremos uma rede geradora, que transforma vetores latentes em saídas de formato `(28, 28, 1)` (representando dígitos MNIST):

In [ ]:
latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

Aqui está a parte chave: o loop de treinamento. Como você pode ver, é bastante simples. A função de passo de treinamento requer apenas 17 linhas.

In [ ]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = keras.optimizers.Adam(learning_rate=0.0003)
g_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

# Instantiate a loss function.
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Decode them to fake images
    generated_images = generator(random_latent_vectors)
    # Combine them with real images
    combined_images = tf.concat([generated_images, real_images], axis=0)

    # Assemble labels discriminating real from fake images
    labels = tf.concat(
        [tf.ones((batch_size, 1)), tf.zeros((real_images.shape[0], 1))], axis=0
    )
    # Add random noise to the labels - important trick!
    labels += 0.05 * tf.random.uniform(labels.shape)

    # Train the discriminator
    with tf.GradientTape() as tape:
        predictions = discriminator(combined_images)
        d_loss = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss, discriminator.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Assemble labels that say "all real images"
    misleading_labels = tf.zeros((batch_size, 1))

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    with tf.GradientTape() as tape:
        predictions = discriminator(generator(random_latent_vectors))
        g_loss = loss_fn(misleading_labels, predictions)
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    return d_loss, g_loss, generated_images


Vamos treinar nossa GAN, chamando repetidamente `train_step` em lotes de imagens.

Já que nosso discriminador e gerador são convnets, você vai querer executar esse código numa GPU.

In [ ]:
import os

# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

epochs = 1  # In practice you need at least 20 epochs to generate nice digits.
save_dir = "./"

for epoch in range(epochs):
    print("\nStart epoch", epoch)

    for step, real_images in enumerate(dataset):
        # Train the discriminator & generator on one batch of real images.
        d_loss, g_loss, generated_images = train_step(real_images)

        # Logging.
        if step % 200 == 0:
            # Print metrics
            print("discriminator loss at step %d: %.2f" % (step, d_loss))
            print("adversarial loss at step %d: %.2f" % (step, g_loss))

            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img" + str(step) + ".png"))

        # To limit execution time we stop after 10 steps.
        # Remove the lines below to actually train the model!
        if step > 10:
            break

É isso! Você vai obter dígitos MNIST falsos bem bonitos depois de cerca de 30 segundos de treinamento na GPU Colab.